# Preprocessing
https://github.com/BrainLesion/preprocessing

## Run Preprocessing

In [1]:
import matplotlib.pyplot as plt
from auxiliary.normalization.percentile_normalizer import PercentileNormalizer
from auxiliary.turbopath import turbopath
from tqdm import tqdm
import os

from brainles_preprocessing.brain_extraction import HDBetExtractor
from brainles_preprocessing.modality import Modality
from brainles_preprocessing.preprocessor import Preprocessor
from brainles_preprocessing.registration import (
    ANTsRegistrator,
    NiftyRegRegistrator
)

In [2]:
def preprocess_exam_in_brats_style(inputDir: str) -> None:
    """
    Perform BRATS (Brain Tumor Segmentation) style preprocessing on MRI exam data.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Raises:
        Exception: If any error occurs during the preprocessing.

    Example:
        brat_style_preprocess_exam("/path/to/exam_directory")

    This function preprocesses MRI exam data following the BRATS style, which includes the following steps:
    1. Normalization using a percentile normalizer.
    2. Registration and correction using NiftyReg.
    3. Brain extraction using HDBet.

    The processed data is saved in a structured directory within the input directory.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Returns:
        None
    """
    inputDir = turbopath(inputDir)
    print("*** start ***")
    brainles_dir = turbopath(inputDir) + "/" + inputDir.name + "_brainles"
    norm_bet_dir = brainles_dir / "normalized_bet"

    t1_file = inputDir.files("*t1.nii.gz")
    t1c_file = inputDir.files("*t1c.nii.gz")
    t2_file = inputDir.files("*t2.nii.gz")
    flair_file = inputDir.files("*fla.nii.gz")

    # we check that we have only one file of each type
    if len(t1_file) == len(t1c_file) == len(t2_file) == len(flair_file) == 1:
        t1File = t1_file[0]
        t1cFile = t1c_file[0]
        t2File = t2_file[0]
        flaFile = flair_file[0]
        
        # normalizer
        percentile_normalizer = PercentileNormalizer(
            lower_percentile=0.1,
            upper_percentile=99.9,
            lower_limit=0,
            upper_limit=1,
        )
        # define modalities
        center = Modality(
            modality_name="t1c",
            input_path=t1cFile,
            normalized_bet_output_path=norm_bet_dir / inputDir.name
            + "_t1c_bet_normalized.nii.gz",
            atlas_correction=True,
            normalizer=percentile_normalizer,
        )

        moving_modalities = [
            Modality(
                modality_name="t1",
                input_path=t1File,
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_t1_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
            Modality(
                modality_name="t2",
                input_path=t2File,
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_t2_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
            Modality(
                modality_name="flair",
                input_path=flaFile,
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_fla_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
        ]

        preprocessor = Preprocessor(
            center_modality=center,
            moving_modalities=moving_modalities,
            #registrator=NiftyRegRegistrator(),
            registrator=ANTsRegistrator(),
            brain_extractor=HDBetExtractor(),
            # optional: we provide a temporary directory as a sandbox for the preprocessin
            temp_folder="/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_directory",
            use_gpu=False,
            #limit_cuda_visible_devices="0",
        )

        preprocessor.run(
            save_dir_coregistration=brainles_dir + "/co-registration",
            save_dir_atlas_registration=brainles_dir + "/atlas-registration",
            save_dir_atlas_correction=brainles_dir + "/atlas-correction",
            save_dir_brain_extraction=brainles_dir + "/brain-extraction",
        )

In [4]:
#EXAMPLE_DATA_DIR = turbopath("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/sample_data")

EXAMPLE_DATA_DIR = turbopath("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/regensburg_sample_data")

exams = EXAMPLE_DATA_DIR.dirs()

for exam in tqdm(exams):
    print("processing:", exam)
    preprocess_exam_in_brats_style(exam)

  0%|          | 0/1 [00:00<?, ?it/s]

processing: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/regensburg_sample_data/regensburg_test_patient
*** start ***
Passed checkpoint 01
Passed checkpoint 02
Passed checkpoint 03
Passed checkpoint 04
File: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_directory/atlas-space/atlas__t1c.nii.gz
preprocessing...
image shape after preprocessing:  (103, 160, 160)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...
Passed checkpoint 05
current image /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_directory/brain-extraction/atlas_bet_t1c.nii.gz
current image /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_directory/brain-extraction/brain_masked/brain_masked__t1.nii.gz
current image /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_directory/brain-extraction/brain_masked/brain_masked__t2.nii.gz
current image /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/preprocessing/temporary_di

100%|██████████| 1/1 [07:44<00:00, 464.91s/it]

Passed checkpoint 06
